In [2]:
import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import EAtools as ea
import pandas as pd
import os   
import sys 
import numpy
import fileinput
import matplotlib.pyplot as plt 
import matplotlib.ticker  
from matplotlib.ticker import FormatStrFormatter 
import pandas.io.sql
import pyodbc 
from matplotlib.pyplot import figure, show
from matplotlib.patches import Ellipse
import numpy as np
from pandas import *
import EAtools as ea
ea.set_options()
ea.ea_report_style()
from pylab import *
import matplotlib.ticker as tkr
from datetime import datetime, date, time, timedelta
ea_p=ea.ea_p
ea_s=ea.ea_s
def cm2inch(value):
    return value/2.54
import pyarrow as pa
import pyarrow.parquet as pq

In [3]:
from db import DB
import os
ea.set_options()
ea.ea_report_style()
%pylab inline
%matplotlib inline
path='/media/usb/notebooks/Julia/20192020_review'
db = DB(profile="hallj") 
# db = DB(profile="millerr") 

Populating the interactive namespace from numpy and matplotlib


/home/julia/.local/lib/python3.5/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['info', 'fft', 'random', 'datetime', 'linalg', 'power']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [4]:
rogersdata = pd.read_parquet('/media/usb/notebooks/Roger/Stacked offer plots/EffectiveEnergyOffers.parquet')                      
rogersdata.head(10)

,Trading_DATE,Trading_Period,Island,Trader_Id,PNode,constr_block,Offer_price,Offer_block,Offer_quantity,Max_Energy,Is_Wind_Offer,FP_cleared_generation,FP_reserve_cleared,rem,Quantity_Calc_Temp,Quantity_Calc
0,2021-10-31,1,NI,GENE,HLY2201 HLY5,FK_HLY2201 HLY5,0.00,1,200.0,381.0,0,270.0,0.0,341.0,200.0,200.0
1,2021-10-31,1,NI,GENE,HLY2201 HLY5,FK_HLY2201 HLY5,0.05,2,15.0,381.0,0,270.0,0.0,341.0,15.0,15.0
2,2021-10-31,1,NI,GENE,HLY2201 HLY5,FK_HLY2201 HLY5,15.05,4,55.0,381.0,0,270.0,0.0,341.0,55.0,55.0
3,2021-10-31,1,NI,GENE,HLY2201 HLY5,FK_HLY2201 HLY5,350.05,5,111.0,381.0,0,270.0,0.0,341.0,111.0,71.0
4,2021-10-31,1,NI,None,ROT1101 WHE0,None,0.00,1,10.0,26.0,0,0.0,0.0,NaN,10.0,10.0
5,2021-10-31,1,NI,None,TGA0331 KMI0,None,0.00,1,15.0,42.0,0,0.0,0.0,NaN,15.0,15.0
6,2021-10-31,1,NI,CTCT,OKI2201 OKI0,None,0.00,1,40.0,40.0,0,40.0,0.0,NaN,40.0,40.0
7,2021-10-31,1,NI,CTCT,PPI2201 PPI0,None,0.00,1,39.0,39.0,0,39.0,0.0,NaN,39.0,39.0
8,2021-10-31,1,NI,CTCT,THI2201 THI1,None,0.00,1,80.0,80.0,0,80.0,0.0,NaN,80.0,80.0
9,2021-10-31,1,NI,CTCT,THI2201 THI2,None,0.00,1,84.0,84.0,0,84.0,0.0,NaN,84.0,84.0


In [37]:
rogersdata.dtypes

Trading_DATE             datetime64[ns]
Trading_Period                    int64
Island                           object
Trader_Id                        object
PNode                            object
constr_block                     object
Offer_price                     float64
Offer_block                       int64
Offer_quantity                  float64
Max_Energy                      float64
Is_Wind_Offer                     int64
FP_cleared_generation           float64
FP_reserve_cleared              float64
rem                             float64
Quantity_Calc_Temp              float64
Quantity_Calc                   float64
dtype: object

In [49]:
rogersdata = rogersdata.rename(columns={'Trading_DATE':'TradingDate','Trading_Period':'TradingPeriod','Offer_block':'Tranche','Quantity_Calc':'Megawatt','Offer_price':'DollarsPerMegawattHour'})
rogersdata['PointOfConnection'] = rogersdata['PNode'].str.slice(0,7)
rogersdata['PointOfConnection'] = rogersdata['PointOfConnection'].str.strip()
rogersdata['Unit'] = rogersdata['PNode'].str.slice(7,13)
rogersdata['Unit'] = rogersdata['Unit'].str.strip()
#rogersdata['Offered_quantity_to_subtract'] = rogersdata['Offer_quantity'] - rogersdata['Quantity_Calc']
rogersdata = rogersdata.sort_values(['TradingDate','TradingPeriod','PointOfConnection','Unit','Tranche'])
rogersdata = rogersdata.reset_index(drop=True).set_index(['TradingDate','TradingPeriod','PointOfConnection','Unit','Tranche'])
#rogersdata = rogersdata[['Offered_quantity_to_subtract']]
rogersdata = rogersdata[['Megawatt','DollarsPerMegawattHour']]
rogersdata.head()

Megawatt  DollarsPerMegawattHour
TradingDate TradingPeriod PointOfConnection Unit Tranche                                  
2014-01-01  1             ARA2201           ARA0 1            22.0                    0.00
                                                 4             4.0                   94.00
                                                 5            52.0                  165.00
                          ARG1101           BRR0 1             6.0                    0.01
                                                 4             5.5                   85.00

In [50]:
temp = rogersdata.reset_index()
temp.Unit.unique()

array(['ARA0', 'BRR0', 'ARI0', 'HBK0', 'ATI0', 'AVI0', 'BEN0', 'WPI0',
       'COB0', 'COL0', 'CYD0', 'GLN0', 'HLY5', 'PTA1', 'PTA2', 'PTA3',
       'WAA0', 'ONU0', 'KAG0', 'KIN0', 'KPI1', 'KPO0', 'KUM0', 'MAN0',
       'ANI0', 'MAT0', 'MHO0', 'MKE1', 'MTI0', 'NAP0', 'NTM0', 'PAE0',
       'OHA0', 'OHB0', 'OHC0', 'OHK0', 'OKI0', 'PPI0', 'WHE0', 'ROX0',
       'RPO0', 'SFD21', 'SFD22', 'TKA1', 'TKB1', 'TKU0', 'KTW0', 'PRI0',
       'TUI0', 'TRC1', 'MOK0', 'WKM0', 'WPA0', 'RKA0', 'TAA0', 'WRK0',
       'WTK0', 'KMI0', 'THI1', 'HLY4', 'OTC0', 'WHI0', 'HLY2', 'SWN5',
       'SPL0', 'SWN0', 'HLY6', 'THI2', 'HLY1', 'TAM0', 'SWN1', 'TWF0',
       'MAH0', 'WHL0', 'NZW0', 'TUK0', 'TAP0', 'MCK0', 'WWD0', 'JRD0',
       'NGB0', 'WPP0'], dtype=object)

In [51]:
temp.PointOfConnection.unique()

array(['ARA2201', 'ARG1101', 'ARI1101', 'ARI1102', 'ASB0661', 'ATI2201',
       'AVI2201', 'BEN2202', 'BWK1101', 'COB0661', 'COL0661', 'CYD2201',
       'GLN0332', 'HLY2201', 'HWA1101', 'HWA1102', 'HWB0331', 'KAW0112',
       'KAW1101', 'KIN0112', 'KPA1101', 'KPO1101', 'KUM0661', 'MAN2201',
       'MAT1101', 'MAT1102', 'MHO0331', 'MKE1101', 'MTI2201', 'NAP2201',
       'NAP2202', 'NSY0331', 'OHA2201', 'OHB2201', 'OHC2201', 'OHK2201',
       'OKI2201', 'PPI2201', 'ROT1101', 'ROX1101', 'ROX2201', 'RPO2201',
       'SFD2201', 'TKA0111', 'TKB2201', 'TKU2201', 'TUI1101', 'TWH0331',
       'WKM2201', 'WPA2201', 'WRK0331', 'WRK2201', 'WTK0111', 'TGA0331',
       'THI2201', 'OTA2202', 'WHI2201', 'SWN2201', 'STK0661', 'KAW0111',
       'BPE0331', 'LTN0331', 'NMA0331', 'TWC2201', 'WDV1101', 'WIL0331',
       'WWD1102', 'WWD1103', 'JRD1101', 'KOE1101', 'WVY1101'],
      dtype=object)

In [52]:
rogersdata.tail()

Megawatt  DollarsPerMegawattHour
TradingDate TradingPeriod PointOfConnection Unit Tranche                                  
2021-06-30  48            WTK0111           WTK0 1          47.000                    0.01
                                                 5           0.000                  485.00
                          WVY1101           WPP0 1          83.347                    0.01
                          WWD1102           WWD0 1          43.599                    0.01
                          WWD1103           WWD0 1          37.944                    0.01

In [53]:
len(rogersdata)

20782122

In [6]:
#match in newpointofconnection and newpointofconnection2
offers = read_csv(path+'/offers_NewPointOfConnectoin.csv')
offers['PointOfConnection'] = offers['PointOfConnection'].str.strip()
offers['Unit'] = offers['Unit'].str.strip()
offers = offers.set_index(['PointOfConnection','Unit'])
offers = offers.groupby(level=[0,1])['NewPointOfConnection','NewPointOfConnection2'].tail(1)
offers.head()

,,NewPointOfConnection,NewPointOfConnection2
PointOfConnection,Unit,,
HLY2201,HLY3,RANKINES,HUNTLY
ARA2201,ARA0,TAUPO,TAUPO
ARI1101,ARI0,TAUPO,TAUPO
ARI1102,ARI0,TAUPO,TAUPO
ATI2201,ATI0,TAUPO,TAUPO


In [7]:
offers

,,NewPointOfConnection,NewPointOfConnection2
PointOfConnection,Unit,,
HLY2201,HLY3,RANKINES,HUNTLY
ARA2201,ARA0,TAUPO,TAUPO
ARI1101,ARI0,TAUPO,TAUPO
ARI1102,ARI0,TAUPO,TAUPO
ATI2201,ATI0,TAUPO,TAUPO
AVI2201,AVI0,WAITAKI,WAITAKI
BEN2202,BEN0,WAITAKI,WAITAKI
CYD2201,CYD0,CYD,CLUTHA
HLY2201,HLY1,RANKINES,HUNTLY


In [55]:
rogersdata = rogersdata.reset_index().set_index(['PointOfConnection','Unit'])
rogersdata = rogersdata.join(offers,how='left')
rogersdata.head()

TradingDate  TradingPeriod  Tranche  Megawatt  DollarsPerMegawattHour NewPointOfConnection NewPointOfConnection2
PointOfConnection Unit                                                                                                                 
ARA2201           ARA0  2014-01-01              1        1      22.0                     0.0                TAUPO                 TAUPO
                  ARA0  2014-01-01              1        4       4.0                    94.0                TAUPO                 TAUPO
                  ARA0  2014-01-01              1        5      52.0                   165.0                TAUPO                 TAUPO
                  ARA0  2014-01-01              2        1      22.0                     0.0                TAUPO                 TAUPO
                  ARA0  2014-01-01              2        4       4.0                    94.0                TAUPO                 TAUPO

In [56]:
rogersdata = rogersdata.reset_index()
rogersdata.head()

,PointOfConnection,Unit,TradingDate,TradingPeriod,Tranche,Megawatt,DollarsPerMegawattHour,NewPointOfConnection,NewPointOfConnection2
0,ARA2201,ARA0,2014-01-01,1,1,22.0,0.0,TAUPO,TAUPO
1,ARA2201,ARA0,2014-01-01,1,4,4.0,94.0,TAUPO,TAUPO
2,ARA2201,ARA0,2014-01-01,1,5,52.0,165.0,TAUPO,TAUPO
3,ARA2201,ARA0,2014-01-01,2,1,22.0,0.0,TAUPO,TAUPO
4,ARA2201,ARA0,2014-01-01,2,4,4.0,94.0,TAUPO,TAUPO


In [57]:
len(rogersdata)

20782122

In [58]:
table = pa.Table.from_pandas(rogersdata)
pq.write_table(table, path + "/rogersdata.parquet")